In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [2]:
import pandas as pd
from pandas.core.common import flatten
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import timeit
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
#importing the file
dfdashboards = pd.read_csv('microstrategy_and_dashboard.csv')
dfmetrics = pd.read_csv('tableau_metric.csv')
dfuserquey = pd.read_csv('sample_user_searchs.csv')

In [4]:
def preprocess_datafiles(dashboard , metric):

    #converting column values to lowercase
    dashboard = dashboard.apply(lambda x: x.astype(str).str.lower())
    metric = metric.apply(lambda x: x.astype(str).str.lower())
    
    #converting global usage to numeric
    dashboard.Global_usage = pd.to_numeric(dashboard.Global_usage)
    
    #splitting caption and descriptors columns 
    dashboard = dashboard[['dashboards','caption','Global_usage']]
    dashboard['metric_names'] = dashboard.caption.str.split('|')
    metric['descriptors'] = metric.descriptors.str.split('|')
    
    #converting metric names and descriptors in each row values to multiple rows

    temp1 = dashboard.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
    temp2 = metric.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})
    
    #merging metrics and descriptors

    master = temp1.merge(temp2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','metric_name','descriptor']]
    master = master.rename(columns={"dashboards":"dashboard_names", "metric_name": "metric_names", "descriptor": "descriptor_names"})

    return master




In [5]:
#"This is the final dataframe we would be using."
master = preprocess_datafiles(dfdashboards,dfmetrics)

In [6]:

dashboard_names_list = list(master.dashboard_names.unique())
metric_names_list = list(master.metric_names.unique())
descriptor_names_list = list(master.descriptor_names.unique())
mastercorpus = dashboard_names_list + metric_names_list + descriptor_names_list

mastercorpus = list(filter(None,mastercorpus))

In [7]:
!pip install ipynb

In [8]:
# from trial2 import fuzzywuzzy_scorers_suggestions
from ipynb.fs.full.trial2 import starts

ModuleNotFoundError: No module named 'ipynb.fs.full.trial2'

In [ ]:
!pip install pynput

In [ ]:
from pynput.keyboard import Listener


In [ ]:
# from pynput.keyboard import Key, Listener
# s = ""
# def on_press(key):
#     print('aaaaaaaaaaaa',type(str(key)))
#     print('{0} pressed'.format(key))
#     key = str(key) + str(key)
#     print(key)
# #     starts(str(key),mastercorpus, master,dashboard_names_list,metric_names_list)
# def on_release(key):
#     print('{0} release'.format(key))
   
#     if key == Key.esc:
#         # Stop listener
#         return False

# # Collect events until dreleasedg
# with Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#     listener.join()

In [ ]:
from pynput.keyboard import Listener, Key

# filename = "key_log.txt"  # The file to write characters to


In [ ]:
buffer = ''

In [ ]:
def on_press(key):
#     f = open(filename, 'a')  # Open the file
    global buffer 
    
    if hasattr(key, 'char'):  # Write the character pressed if available
        x = (key.char)
    elif key == Key.space:  # If space was pressed, write a space
        x = (' ')
    elif key == Key.enter:  # If enter was pressed, write a new line
        pass
    elif key == Key.tab:  # If tab was pressed, write a tab
        pass
    elif key == Key.esc:
        # Stop listener
        return False
    else:  # If anything else was pressed, write [<key_name>]
        x = ('[' + key.name + ']')
    
    buffer += x 
    
    print(type(buffer))
    print(buffer)
    starts(str(buffer),mastercorpus, master,dashboard_names_list,metric_names_list)



In [ ]:
with Listener(on_press=on_press) as listener:  # Setup the listener
    listener.join()  # Join the  thread to the main thread

In [ ]:
amber ale